In [1]:
import os
import pyspark
conf = pyspark.SparkConf()
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

In [2]:
spark

In [3]:
df = spark.read.csv("Electric_Vehicle_Population_Data.csv")

In [4]:
df

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string]

In [5]:
df1 = spark.read.option("header", True).option("InferSchema", True).csv("Electric_Vehicle_Population_Data.csv")

In [6]:
df1

DataFrame[VIN (1-10): string, County: string, City: string, State: string, Postal Code: int, Model Year: int, Make: string, Model: string, Electric Vehicle Type: string, Clean Alternative Fuel Vehicle (CAFV) Eligibility: string, Electric Range: int, Base MSRP: int, Legislative District: int, DOL Vehicle ID: int, Vehicle Location: string, Electric Utility: string, 2020 Census Tract: bigint]

In [7]:
df1.show()

+----------+-----------+-----------+-----+-----------+----------+---------+-------+---------------------+-------------------------------------------------+--------------+---------+--------------------+--------------+--------------------+--------------------+-----------------+
|VIN (1-10)|     County|       City|State|Postal Code|Model Year|     Make|  Model|Electric Vehicle Type|Clean Alternative Fuel Vehicle (CAFV) Eligibility|Electric Range|Base MSRP|Legislative District|DOL Vehicle ID|    Vehicle Location|    Electric Utility|2020 Census Tract|
+----------+-----------+-----------+-----+-----------+----------+---------+-------+---------------------+-------------------------------------------------+--------------+---------+--------------------+--------------+--------------------+--------------------+-----------------+
|5YJ3E1EA8J|  San Diego|  Oceanside|   CA|      92051|      2018|    TESLA|MODEL 3| Battery Electric ...|                             Clean Alternative...|           215

In [8]:
df1.sort("City").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [City#70 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(City#70 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=58]
      +- FileScan csv [VIN (1-10)#68,County#69,City#70,State#71,Postal Code#72,Model Year#73,Make#74,Model#75,Electric Vehicle Type#76,Clean Alternative Fuel Vehicle (CAFV) Eligibility#77,Electric Range#78,Base MSRP#79,Legislative District#80,DOL Vehicle ID#81,Vehicle Location#82,Electric Utility#83,2020 Census Tract#84L] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/G:/My Drive/CS-GY 6513 Big Data/Big-Data-Code/Spark/Electric_Veh..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<VIN (1-10):string,County:string,City:string,State:string,Postal Code:int,Model Year:int,Ma...




In [9]:
df1.select("City").show()

+-----------+
|       City|
+-----------+
|  Oceanside|
|      Derby|
| Marysville|
|  Bremerton|
|    Edmonds|
|Walla Walla|
|    Everett|
| Oak Harbor|
|    Bothell|
|    Olympia|
|  Anacortes|
|    Bothell|
|       Kent|
|    Bothell|
|    Olympia|
|  Bremerton|
|     Olalla|
|    Woodway|
|    Bothell|
|    Pullman|
+-----------+
only showing top 20 rows



In [10]:
df1.createOrReplaceTempView("cars")

In [11]:
spark.sql("select City from cars").show()

+-----------+
|       City|
+-----------+
|  Oceanside|
|      Derby|
| Marysville|
|  Bremerton|
|    Edmonds|
|Walla Walla|
|    Everett|
| Oak Harbor|
|    Bothell|
|    Olympia|
|  Anacortes|
|    Bothell|
|       Kent|
|    Bothell|
|    Olympia|
|  Bremerton|
|     Olalla|
|    Woodway|
|    Bothell|
|    Pullman|
+-----------+
only showing top 20 rows



In [12]:
spark.sql("select City from cars").explain()

== Physical Plan ==
FileScan csv [City#70] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/G:/My Drive/CS-GY 6513 Big Data/Big-Data-Code/Spark/Electric_Veh..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<City:string>




In [13]:
df1.select("City").explain()

== Physical Plan ==
FileScan csv [City#70] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/G:/My Drive/CS-GY 6513 Big Data/Big-Data-Code/Spark/Electric_Veh..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<City:string>




In [14]:
df1.groupBy("County").count().show()

+---------------+-----+
|         County|count|
+---------------+-----+
|       Thurston| 4326|
|          Ferry|   27|
|          Marin|    1|
|       Kittitas|  409|
|Prince George's|    3|
|      Snohomish|13293|
|       Franklin|  387|
|         Kitsap| 4016|
|           Bell|    2|
|       Arapahoe|    1|
|     Alexandria|    5|
|    Los Angeles|    8|
|       Okanogan|  154|
|     Sacramento|    1|
|      San Diego|   23|
|           null|    2|
|   Grays Harbor|  415|
|     Chesapeake|    4|
|      St. Louis|    1|
|           King|62375|
+---------------+-----+
only showing top 20 rows



In [15]:
from pyspark.sql.functions import max
df1.select(max("Electric Range")).show()

+-------------------+
|max(Electric Range)|
+-------------------+
|                337|
+-------------------+



In [16]:
from pyspark.sql.functions import desc

df1.sort(desc("Electric Range")).select("county","city","electric range", "model").show(3)

+---------+---------+--------------+-------+
|   county|     city|electric range|  model|
+---------+---------+--------------+-------+
|    Clark|Vancouver|           337|MODEL S|
|     King|Carnation|           337|MODEL S|
|Snohomish|  Bothell|           337|MODEL S|
+---------+---------+--------------+-------+
only showing top 3 rows



In [17]:
df1.groupBy("Model").max("Electric Range")\
.withColumnRenamed("Model", "Car Model").withColumnRenamed("max(Electric Range)","Highest Range")\
.sort(desc("Highest Range")).show()

+----------------+-------------+
|       Car Model|Highest Range|
+----------------+-------------+
|         MODEL S|          337|
|         MODEL 3|          322|
|         MODEL X|          293|
|         MODEL Y|          291|
|         BOLT EV|          259|
|            KONA|          258|
|        ROADSTER|          245|
|            NIRO|          239|
|          I-PACE|          234|
|             PS2|          233|
|          E-TRON|          222|
|E-TRON SPORTBACK|          218|
|            LEAF|          215|
|          TAYCAN|          203|
|           IONIQ|          170|
|              I3|          153|
|          E-GOLF|          125|
|         SOUL EV|          111|
|         HARDTOP|          110|
|            RAV4|          103|
+----------------+-------------+
only showing top 20 rows



In [18]:
df1.printSchema()

root
 |-- VIN (1-10): string (nullable = true)
 |-- County: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Model Year: integer (nullable = true)
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Electric Vehicle Type: string (nullable = true)
 |-- Clean Alternative Fuel Vehicle (CAFV) Eligibility: string (nullable = true)
 |-- Electric Range: integer (nullable = true)
 |-- Base MSRP: integer (nullable = true)
 |-- Legislative District: integer (nullable = true)
 |-- DOL Vehicle ID: integer (nullable = true)
 |-- Vehicle Location: string (nullable = true)
 |-- Electric Utility: string (nullable = true)
 |-- 2020 Census Tract: long (nullable = true)



In [19]:
df1.select("Model").show()

+-------+
|  Model|
+-------+
|MODEL 3|
| FUSION|
|   LEAF|
|     I3|
|MODEL 3|
|BOLT EV|
|   NIRO|
|   LEAF|
|MODEL 3|
|MODEL 3|
|MODEL 3|
|MODEL 3|
|MODEL 3|
|MODEL S|
|MODEL 3|
| OPTIMA|
|SOUL EV|
|   LEAF|
|  C-MAX|
|MODEL Y|
+-------+
only showing top 20 rows



In [20]:
df2 = spark.read.\
format("json").load("flight-data/json/2015-summary.json")
df2.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [21]:
df2 = spark.read.json("flight-data/json/2015-summary.json")
df2.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [ ]:
from pyspark.sql.functions import col
df1.col("cars")